<img src="https://teaching.bowyer.ai/sdsai/resources/0/img/IMPERIAL_logo_RGB_Blue_2024.svg" alt="Imperial Logo" width="500"/><br /><br />

Neural Networks and Deep Learning - Tutorial Exercise Solutions
==============
### SURG70098 - Surgical Data Science and AI
### Stuart Bowyer

# Setup

In [ ]:
# Install and import
%pip install pandas
%pip install matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import pandas_gbq

# @markdown Enter your Google Cloud Project ID:
project_id = 'mimic-test-476513'  # @param {type:"string"}

df_day1_vitalsign = pandas_gbq.read_gbq("""
  SELECT *
  FROM `physionet-data.mimiciv_3_1_derived.first_day_vitalsign`
  LEFT JOIN (
    SELECT
      subject_id,
      stay_id,
      gender,
      race,
      admission_age,
      dod IS NOT NULL AS mortality
    FROM
      `physionet-data.mimiciv_3_1_derived.icustay_detail`
  )
  USING(subject_id, stay_id)
  LEFT JOIN (
    SELECT
      stay_id,
      AVG(weight) as weight
    FROM
      `physionet-data.mimiciv_3_1_derived.weight_durations`
    GROUP BY
      stay_id
  )
  USING(stay_id)
  LEFT JOIN (
    SELECT
      stay_id,
      CAST(AVG(height) AS FLOAT64) AS height
    FROM
      `physionet-data.mimiciv_3_1_derived.height`
    GROUP BY
      stay_id
  )
  USING(stay_id)
  WHERE heart_rate_mean IS NOT NULL
  LIMIT 100000
""", project_id=project_id)

# Exercise 5.1


In [ ]:
w1 = 0.5
w2 = 1.7
w3 = 0.001
bias = -52

def relu(x):
    if (x < 0):
        return(0)
    return(x)

va1 = 65
va2 = 1.7
va3 = 8200

vb1 = 82
vb2 = 1.85
vb3 = 9800

output_a = relu(w1 * va1 + w2 * va2 + w3 * va3 + bias)
output_b = relu(w1 * vb1 + w2 * vb2 + w3 * vb3 + bias)

print(f'Neuron output for A is {output_a}')
print(f'Neuron output for B is {output_b}')


Neuron output for A is 0
Neuron output for B is 1.9450000000000074


# Exercise 5.2



In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, roc_auc_score
from imblearn.over_sampling import RandomOverSampler

# Prepare our data
data = df_day1_vitalsign.dropna(subset=['admission_age', 'heart_rate_mean', 'sbp_mean', 'glucose_mean', 'mortality'])
X = data[['admission_age', 'heart_rate_mean', 'sbp_mean', 'glucose_mean']]
Y = data['mortality']

# Create the Neural network with 20 neurons in the hidden layer
model = MLPClassifier(
    hidden_layer_sizes=(20),
    random_state=1
)

# Train Test Split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

# Train the model
model.fit(X_train, Y_train)

# Test the model
Y_pred = model.predict(X_test)

# Metrics
print(f"Confusion Matrix:\n {confusion_matrix(Y_test, Y_pred)}")
print(f"Accuracy: {accuracy_score(Y_test, Y_pred)}")
print(f"f1-score: {f1_score(Y_test, Y_pred)}")
print(f"ROC AUC:  {roc_auc_score(Y_test, Y_pred)}")

# Cross validate to assess the balanced performance
nn_scores = cross_validate(model, X, Y, cv=5, scoring=['f1','accuracy','precision','recall','roc_auc'])

print(f'N:                              {len(Y)}')
print(f'Cross-Validation Accuracy Mean: {nn_scores["test_accuracy"].mean()}')
print(f'Cross-Validation F1 Mean:       {nn_scores["test_f1"].mean()}')
print(f'Cross-Validation ROC AUC Mean:  {nn_scores["test_roc_auc"].mean()}')